Add gain method to CrystalSlice class that applies Frantz-Nodvik equation: issue #137
https://github.com/radiasoft/rslaser/issues/137
____________

Continue with work in only one prop_type of CrystalSlice.propagate(): 'n0n2_srw'.

_________________
Imports

In [7]:
import sys, time
import math
import numpy as np
from pykern import pkcli
from pykern.pkcollections import PKDict

# The rslaser library may not be installed, so a check is required.
try:
    import rslaser
except:
    # Developers should use 'pip install -e .' from the command line.
    # Users can install directly from GitHub --
    !{sys.executable} -m pip install git+https://github.com/radiasoft/rslaser.git
    import rslaser

from rslaser.pulse import pulse
from rslaser.optics import element
from rslaser.optics import drift
from rslaser.optics import crystal

import scipy.constants as const

import srwlib
from srwlib import srwl

# # 2D plotting
# import matplotlib as mpl
# import matplotlib.pyplot as plt
# from matplotlib import cm

# # reset the notebook style
# mpl.rcParams.update(mpl.rcParamsDefault)
# %matplotlib inline

______________
Set laser defaults

In [8]:
# specify parameters
_LASER_PULSE_DEFAULTS = PKDict(
        photon_e_ev=1.5498, # Photon energy [eV], calculated from 800nm wavelength
        nslice = 3,
        nx_slice = 32,
        ny_slice = 32,
)

# Instantiate the default parameters
params = _LASER_PULSE_DEFAULTS.copy()

______________
Define a simple lattice

In [9]:
# instantiate a drift
L_drift = 0.50  # [m]
e_drift = drift.Drift(L_drift)

# instantiate the crystal
crystal_params = PKDict(length = 0.01,  # [m]
                        nslice = 1,
                        )
e_crystal = crystal.Crystal(crystal_params)

___________
Instantiate the laser pulse

In [10]:
thisPulse = pulse.LaserPulse(params)

______________
Propagate and calculate

In [11]:
# First drift
thisPulse = e_drift.propagate(thisPulse, 'default')

# Through crystal (1 slice for now)
thisPulse = e_crystal.propagate(thisPulse, 'n0n2_srw')

# Last drift
thisPulse = e_drift.propagate(thisPulse, 'default')


# Sorting out initial pop_inversion_mesh

# # Calculate change in photon number

# cslice_length = e_crystal.length / e_crystal.nslice  # [m]
# absorp_cross_sec = 2.0e-24                           # [m^2] (2.0e-20 cm^2)
# degen_factor = 1.0                                   # Not sure what this value should be
# pop_inversion = e_crystal.slice[0].pop_inversion_mesh

# wfr0 = thisPulse.slice[0].wfr
# dx = (wfr0.mesh.xFin - wfr0.mesh.xStart)/wfr0.mesh.nx
# dy = (wfr0.mesh.yFin - wfr0.mesh.yStart)/wfr0.mesh.ny

# # total number of incident photons per unit area
# n_incident_photons = thisPulse.slice[0].n_photons_2d / (dx * dy)   # [1/m^2]
# print(np.max(thisPulse.slice[0].n_photons_2d))

# # Calcualte the gain
# energy_gain = (1.0 / (degen_factor * absorp_cross_sec * n_incident_photons)) * \
#                 np.log(1 + np.exp(absorp_cross_sec * pop_inversion * cslice_length) * \
#                 (np.exp(degen_factor * absorp_cross_sec * n_incident_photons) - 1.0))

# # Update the number of photons
# new_n_photons = thisPulse.slice[0].n_photons_2d * energy_gain
# print(np.max(new_n_photons))

prop_type = n0n2_srw
n0: 1.75, n2: 1.75
Propagated pulse slice  1  of  3
Propagated pulse slice  2  of  3
Propagated pulse slice  3  of  3


In [12]:
# # Update the pop_inversion
# print(np.max(pop_inversion))
# change_pi = pop_inversion - (degen_factor * n_incident_photons * (energy_gain - 1.0) / cslice_length)
# print(np.max(change_pi))
# pop_inversion -= degen_factor * n_incident_photons * (energy_gain - 1.0) / cslice_length
# print(np.max(pop_inversion))

1. Put the calculations into the notebook and element.py
2. Fix the pop_inversion initialization